# Libraries

In [1]:
import os
nnn = 1
os.environ["OMP_NUM_THREADS"] = str(nnn) # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = str(nnn) # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = str(nnn) # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = str(nnn) # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = str(nnn)  # export NUMEXPR_NUM_THREADS=1

In [2]:
from TELF.factorization.HNMFk import HNMFk
from TELF.pre_processing import Beaver
from TELF.pre_processing.Vulture.tokens_analysis.top_words import get_top_words

In [3]:
import pandas as pd
import numpy as np
import scipy.sparse

# Load Data

In [4]:
df = pd.read_csv("../../data/sample.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   eid             940 non-null    object
 1   title           940 non-null    object
 2   year            940 non-null    int64 
 3   abstract        940 non-null    object
 4   authors         940 non-null    object
 5   author_ids      940 non-null    object
 6   references      843 non-null    object
 7   clean_abstract  940 non-null    object
dtypes: int64(1), object(7)
memory usage: 58.9+ KB


# Build Matrix

In [5]:
beaver = Beaver()

beaver_vocab_settings = {
    "dataset":df,
    "target_column":"clean_abstract",
    "min_df":10,
    "max_df":0.5,
}

vocabulary = beaver.get_vocabulary(**beaver_vocab_settings)
len(vocabulary)

467

In [6]:
vocabulary[:5]

array(['128pb', '2018', '32x', 'ability', 'abstain'], dtype=object)

In [7]:
beaver_matrix_settings = {
    "dataset":df,
    "target_column":"clean_abstract",
    "options":{"min_df": 5, "max_df": 0.5, "vocabulary":vocabulary},
    "matrix_type":"tfidf",
    "save_path":None
}

X, _ = beaver.documents_words(**beaver_matrix_settings)

In [8]:
# put the samples to columns and features (tokens) to rows
X = X.T.tocsr()
X.shape

(467, 940)

In [9]:
X

<467x940 sparse matrix of type '<class 'numpy.float32'>'
	with 72655 stored elements in Compressed Sparse Row format>

In [10]:
assert X.shape[1] == len(df)

# Custom Callback Class

In [11]:
class CustomSemanticCallback:

    def __init__(self, 
                 df:pd.DataFrame, 
                 vocabulary:list,
                 target_column="clean_abstract",
                 options={"min_df": 5, "max_df": 0.5},
                 matrix_type="tfidf") -> None:
        #
        # Data
        #
        self.df = df
        self.vocabulary = vocabulary
        
        #
        # Beaver Settings
        #
        self.target_column = target_column
        # use the same vocabulary on each NMFk decomposition
        options["vocabulary"] = self.vocabulary
        self.options = options
        self.matrix_type = matrix_type
    
    def __call__(self, original_indices:np.ndarray) -> scipy.sparse.csr_matrix:
        
        current_beaver = Beaver()
        current_df = self.df.iloc[original_indices].copy()
        current_beaver_matrix_settings = {
            "dataset":current_df,
            "target_column":self.target_column,
            "options":self.options,
            "matrix_type":self.matrix_type,
            "save_path":None
        }
        current_X, vocab = current_beaver.documents_words(**current_beaver_matrix_settings)
        # put the samples to columns and features (tokens) to rows
        current_X = current_X.T.tocsr()
        # below dictionary can be used to save additional information in each node
        save_at_node = {"vocab":vocab}
        return current_X, save_at_node

In [12]:
custom_callback = CustomSemanticCallback(df=df, vocabulary=vocabulary)

# Settings

In [13]:
Ks = np.arange(1, 11, 1)
perts = 2
iters = 200
eps = 0.005
init = "nnsvd"
save_path = "Semantic_HNMFk_results_path"
name = "example_Semantic_HNMFk"

In [14]:
nmfk_params = {
    "n_perturbs":perts,
    "n_iters":iters,
    "epsilon":eps,
    "n_jobs":-1,
    "init":init, 
    "use_gpu":False,
    "save_path":save_path, 
    "predict_k_method":"sill",
    "verbose":False,
    "nmf_verbose":False,
    "transpose":False,
    "sill_thresh":0.8,
    "pruned":True,
    'nmf_method':'nmf_fro_mu',
    "calculate_error":False,
    "use_consensus_stopping":0,
    "calculate_pac":False,
    "consensus_mat":False,
    "perturb_type":"uniform",
    "perturb_multiprocessing":False,
    "perturb_verbose":False,
    "simple_plot":True
}

In [15]:
hnmfk_params = {
    # This can be used to re-generate the data matrix X before each NMFk operation. 
    # When not used, slice of original X is taken, which is equal to serial decomposition.
    # ``generate_X_callback`` object should be a class with ``def __call__(original_indices)`` 
    # defined so that ``new_X, save_at_node=generate_X_callback(original_indices)`` can be done.
    # ``original_indices`` hyper-parameter is the indices of samples (columns of original X when clustering on H).
    # Here ``save_at_node`` is a dictionary that can be used to save additional information in each 
    # node's ``user_node_data`` variable. 
    "generate_X_callback":custom_callback,
    # we can specify nmfk parameters for each depth, or use same for all depth
    # below will use the same nmfk parameters for all depths
    # when using for each depth, append to the list 
    # for example, [nmfk_params0, nmfk_params1, nmfk_params2] for depth of 2
    "nmfk_params": [nmfk_params], 
    # where to perform clustering, can be W or H
    # if W, row of X should be samples
    # if H, columns of X should be samples
    "cluster_on":"H",
    # how deep to go in each topic after root node
    # if -1, it goes until samples cannot be seperated further
    "depth":2,
    # stopping criteria for num of samples
    "sample_thresh":5,
    # if K2=True, decomposition is done only for k=2 instead of 
    # finding and predicting the number of stable latent features
    "K2":False,
    # after first nmfk, when selecting Ks search range, minimum k to start
    "Ks_deep_min":1,
    # After first nmfk, when selecting Ks search range, maximum k to try.
    # When None, maximum k will be same as k selected for parent node.
    "Ks_deep_max": None,
    # after first nmfk, when selecting Ks search range, k step size
    "Ks_deep_step":1,
    # where to save
    "experiment_name":name
}

# Run HNMFk

In [16]:
model = HNMFk(**hnmfk_params)
model.fit(X, Ks)

/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'time': 5.928023099899292}

In [17]:
model.node_save_paths

{'d7258594-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk/depth_0/d7258594-fd0f-11ee-bff4-ea13c6004852/node_d7258594-fd0f-11ee-bff4-ea13c6004852.p',
 'd7d13aa6-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk/depth_1/d7d13aa6-fd0f-11ee-bff4-ea13c6004852/node_d7d13aa6-fd0f-11ee-bff4-ea13c6004852.p',
 'd7d13bbe-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk/depth_1/d7d13bbe-fd0f-11ee-bff4-ea13c6004852/node_d7d13bbe-fd0f-11ee-bff4-ea13c6004852.p',
 'd7d13c72-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk/depth_1/d7d13c72-fd0f-11ee-bff4-ea13c6004852/node_d7d13c72-fd0f-11ee-bff4-ea13c6004852.p',
 'd7d13cc2-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk/depth_1/d7d13cc2-fd0f-11ee-bff4-ea13c6004852/node_d7d13cc2-fd0f-11ee-bff4-ea13c6004852.p',
 'd7d13cfe-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk/depth_1/d7d13cfe-fd0f-11ee-bff4-ea13c6004852/node_d7d13cfe-fd0f-11ee-bff4-ea13c6004852.p',
 'd7d13d4e-fd0f-11ee-bff4-ea13c6004852': 'example_Semantic_HNMFk

# Example Traverse Graph

 Below functions can be utilized to walk the graph:

 ```python
model.traverse_nodes()
model.go_to_root()
model.get_node()
model.go_to_parent()
model.go_to_children(idx:int)
 ```

 We can reset the iterator to go back to the root node as shown below:

In [18]:
node = model.go_to_root()
node["node_name"]

'd7258594-fd0f-11ee-bff4-ea13c6004852'

 HNMFk class includes a iterator that enables walking the graph nodes. Current node the iterator is at can be obtained as shown below (always starts at root node):

In [19]:
node = model.get_node()
node.keys()

dict_keys(['node_name', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data'])

 We can also see the name of the node:

In [20]:
node["node_name"]

'd7258594-fd0f-11ee-bff4-ea13c6004852'

 And we can see the child nodes:

In [21]:
node["child_node_names"]

['d7d13aa6-fd0f-11ee-bff4-ea13c6004852',
 'd7d13bbe-fd0f-11ee-bff4-ea13c6004852',
 'd7d13c72-fd0f-11ee-bff4-ea13c6004852',
 'd7d13cc2-fd0f-11ee-bff4-ea13c6004852',
 'd7d13cfe-fd0f-11ee-bff4-ea13c6004852',
 'd7d13d4e-fd0f-11ee-bff4-ea13c6004852',
 'd7d13d9e-fd0f-11ee-bff4-ea13c6004852',
 'd7d13e0c-fd0f-11ee-bff4-ea13c6004852',
 'd7d13e52-fd0f-11ee-bff4-ea13c6004852']

 We can go to the child node specified with an index. For example, to go to the first child, we index at 0. When we go to the child node, it will return the child node and set the iterator to the child node.

In [22]:
node = model.go_to_children(1)
node["node_name"]

'd7d13bbe-fd0f-11ee-bff4-ea13c6004852'

In [23]:
node.keys()

dict_keys(['node_name', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data'])

 Take a look at the parent node, which should be the root:

In [24]:
node["parent_node_name"]

'd7258594-fd0f-11ee-bff4-ea13c6004852'

get_node() always returns the node we are currently at:

In [25]:
node = model.get_node()
node["node_name"]

'd7d13bbe-fd0f-11ee-bff4-ea13c6004852'

Go back to parent:

In [26]:
node = model.go_to_parent()
node["node_name"]

'd7258594-fd0f-11ee-bff4-ea13c6004852'

 From each node, we can get the samples that was clustered in the node:

In [27]:
node["original_indices"][:5], len(node["original_indices"])

(array([0, 1, 2, 3, 4]), 940)

We can also check if a given node in the graph a leaf node

In [28]:
node["leaf"]

False

 Finally, we can obtain all the nodes using the following method. Note that while other other node iterator options above are online, meaning each node is loaded into memory one at a time, the following traversal will load all nodes into the memory:

In [30]:
all_nodes = model.traverse_nodes()
len(all_nodes)

10

In [31]:
all_nodes[0].keys()

dict_keys(['node_name', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data'])

# Organize Leaf Nodes

In [33]:
leaf_nodes = []

for node in all_nodes:
    if node["leaf"]:
        current_documents = df.iloc[node["original_indices"]]
        top_1grams = get_top_words(current_documents["clean_abstract"].to_dict(), top_n=50, n_gram=1, verbose=False, filename=None)
        top_2grams = get_top_words(current_documents["clean_abstract"].to_dict(), top_n=50, n_gram=2, verbose=False, filename=None)

        leaf_nodes.append({   
            "node_name":node["node_name"], 
            "depth":node["depth"],
            "W":node["W"],
            "H":node["H"],
            "user_node_data":node["user_node_data"],
            "parent_node_name":node["parent_node_name"], 
            "child_node_names":node["child_node_names"], 
            "num_samples":node["num_samples"], 
            "sample_indices":node["original_indices"],
            "documents":current_documents,
            "top_1grams":top_1grams,
            "top_2grams":top_2grams,
        })
        
len(leaf_nodes)

9

### Look at top 2-grams in each leaf node

In [34]:
leaf_nodes[0]["top_2grams"].head(10)

,word,tf,df,df_fraction,tf_fraction
0,malware family,450,90,1.0,1.778656
1,malware dna,360,90,1.0,1.422925
2,novel malware,270,90,1.0,1.067194
3,generate malware,180,90,1.0,0.711462
4,rare prominent,180,90,1.0,0.711462
5,prominent malware,180,90,1.0,0.711462
6,malware sample,180,90,1.0,0.711462
7,specimen belong,180,90,1.0,0.711462
8,classify malware,180,90,1.0,0.711462
9,family benign,180,90,1.0,0.711462


In [35]:
leaf_nodes[1]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,truncate singular,202,101,1.0,1.90566
1,implementation svd,202,101,1.0,1.90566
2,singular vector,202,101,1.0,1.90566
3,memory complex,202,101,1.0,1.90566
4,latency associate,202,101,1.0,1.90566


In [36]:
leaf_nodes[2]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,malware family,354,118,1.0,6.555556
1,novel malware,236,118,1.0,4.370370
2,malware benign,236,118,1.0,4.370370
3,benign ware,236,118,1.0,4.370370
4,ware classification,236,118,1.0,4.370370


In [37]:
leaf_nodes[3]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,build dataset,297,99,1.0,3.621951
1,dataset scientific,198,99,1.0,2.414634
2,scientific literature,198,99,1.0,2.414634
3,topic model,198,99,1.0,2.414634
4,machine learn,198,99,1.0,2.414634


In [38]:
leaf_nodes[4]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,shoot federate,210,105,1.0,2.560976
1,federate learn,210,105,1.0,2.560976
2,item pattern,210,105,1.0,2.560976
3,negative matrix,105,105,1.0,1.280488
4,matrix factorization,105,105,1.0,1.280488


In [39]:
leaf_nodes[5]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,scada system,452,113,1.0,3.074830
1,tensor decomposition,339,113,1.0,2.306122
2,grid system,226,113,1.0,1.537415
3,anomaly scada,226,113,1.0,1.537415
4,expect behavior,226,113,1.0,1.537415


In [40]:
leaf_nodes[6]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,sub topic,500,100,1.0,3.311258
1,topic model,300,100,1.0,1.986755
2,hierarchical senmfk,300,100,1.0,1.986755
3,text document,200,100,1.0,1.324503
4,document matrix,200,100,1.0,1.324503


In [41]:
leaf_nodes[7]["top_2grams"].head(5)

,word,tf,df,df_fraction,tf_fraction
0,sparse matrix,232,116,1.0,2.148148
1,matrix operation,232,116,1.0,2.148148
2,multi gpu,232,116,1.0,2.148148
3,latency associate,232,116,1.0,2.148148
4,collective communication,232,116,1.0,2.148148


In [42]:
leaf_nodes[8]["top_2grams"].head(10)

,word,tf,df,df_fraction,tf_fraction
0,malware family,490,98,1.0,4.298246
1,semi supervise,294,98,1.0,2.578947
2,class imbalance,196,98,1.0,1.719298
3,hnmfk classify,196,98,1.0,1.719298
4,identification family,98,98,1.0,0.859649
5,family malware,98,98,1.0,0.859649
6,malware specimen,98,98,1.0,0.859649
7,specimen belong,98,98,1.0,0.859649
8,belong essential,98,98,1.0,0.859649
9,essential understand,98,98,1.0,0.859649
